In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DMEIRL
from trajGen_grid_world import GridWorld_trajGen
from utils import utils

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

no background image


In [ ]:
reward_mat = np.array([[ 1, 1, 1, 1, 1, 1, 1, 1, 2, 2],
                       [-1,-1, 1, 1, 1, 1, 1, 1, 1, 2],
                       [-2,-1,-1,-1,-1,-1,-1, 1, 1, 1],
                       [-2,-2,-2,-2,-2,-2,-2,-1,-1,-1],
                       [-1,-1,-1,-1,-1, 1,-2,-2,-2,-2],
                       [ 0, 0, 0, 1, 1, 2, 1,-1,-1,-1],
                       [ 0, 0, 1, 1, 2, 2, 2, 1,-1,-1],
                       [ 1, 1, 1, 1, 1, 1, 1, 0,-1,-1],
                       [ 2, 1, 1, 1, 1, 1, 0, 0,-1,-1],
                       [ 2, 2, 1, 1, 1, 1, 0,-1,-1,-2],])

In [ ]:
reward_mat = np.array([[ 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
                       [-1,-1, 1, 1, 1, 1, 1, 1, 1, 1],
                       [-1,-1,-1, 1, 1, 1, 1, 1, 1, 1],
                       [-1,-1,-1, 1, 1, 1,-1,-1,-1,-1],
                       [-1,-1,-1,-1, 1, 1,-1,-1,-1,-1],
                       [ 1, 1, 1, 1, 1, 1, 1,-1,-1,-1],
                       [ 1, 1, 1, 1, 1, 1, 1, 1,-1,-1],
                       [-1,-1,-1,-1,-1,-1,-1, 0,-1,-1],
                       [ 0,-1,-1,-1,-1,-1, 0, 0,-1,-1],
                       [ 0, 0,-1,-1,-1,-1, 0,-1,-1,-1],])

In [117]:
reward_mat = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0,-1,-1,-1,-1, 0, 0, 0, 0],
                       [ 0, 0,-1,-1,-1,-1, 0, 0, 0, 0],
                       [ 0, 0,-1,-1,-1,-1, 1, 1, 1, 1],
                       [ 0, 0,-1,-1,-1,-1, 1, 1, 1, 1],
                       [ 0, 0,-1,-1,-1,-1, 1, 1, 1, 1],
                       [ 0, 0,-1,-1,-1,-1, 1, 1, 1, 1],
                       [ 0, 0,-1,-1,-1,-1, 1, 1, 1, 1],])
#reward_mat = utils.Normalize_2DArr(reward_mat)
world_trajGen = GridWorld_trajGen(10,10,reward_mat,
                                  deactive_states=[80,81,82,83,84,
                                                   70,71,72,73,74,75,
                                                   60,61,62,63,64,
                                                   56,57,58,59,
                                                   47,48,49,
                                                   38,39,
                                                   ],
                                  trans_prob=0.8,discount=0.9)
world_trajGen.ShowRewards(title='真实奖励')

In [118]:
world_trajGen.GenerateTrajectories(traj_count=500,traj_length=30,save=True)

100%|██████████| 500/500 [00:00<00:00, 1744.79it/s]


,m,trajs
0,1,"[(14, 4, 15), (15, 4, 16), (16, 1, 6), (6, 1, ..."
1,2,"[(79, 1, 79), (79, 1, 89), (89, 0, 89), (89, 0..."
2,3,"[(86, 4, 87), (87, 4, 88), (88, 4, 89), (89, 0..."
3,4,"[(3, 4, 4), (4, 4, 5), (5, 1, 15), (15, 4, 16)..."
4,5,"[(33, 2, 23), (23, 4, 24), (24, 4, 25), (25, 4..."
...,...,...
495,496,"[(67, 4, 68), (68, 4, 69), (69, 1, 69), (69, 1..."
496,497,"[(76, 4, 77), (77, 4, 78), (78, 4, 79), (79, 1..."
497,498,"[(50, 2, 50), (50, 2, 40), (40, 2, 30), (30, 2..."
498,499,"[(7, 1, 17), (17, 1, 27), (27, 1, 37), (37, 3,..."


In [119]:
traj_path = r'demo_expert_trajs_0110.csv'
world = GridWorld(
    states_features=world_trajGen.states_features,
    expert_traj_filePath=traj_path,
    width=10,
    height=10,
    active_all=False
)
world.ShowGridWorld_Count_log(title='轨迹计数分布(对数)')

In [120]:
world.ShowGridWorld_Activated()

In [121]:
dme = DMEIRL(world,layers = (16,16),lr = 0.001)

In [122]:
rewards_result = dme.train(n_epochs=300,demo=True)

100%|██████████| 300/300 [18:48<00:00,  3.76s/it]


In [123]:
len(rewards_result)

75

In [124]:
world.ShowRewardsResult(rewards_result,title='DMEIRL奖励复原结果')

In [125]:
world.ShowRewardsAnimation(dme.rewards_list)